<a href="https://colab.research.google.com/github/Saisrujan4/NNDL/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
import numpy as np
import math as mt
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [80]:
data=pd.read_csv('/content/drive/MyDrive/LD.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
x = []
y = []
for i in range(24 * 7, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i - 24 * 7])
    y.append(data['Load (kW)'].iloc[i])


In [82]:
data = pd.DataFrame({'Previous Week Load' : x, 'Present Week Load' : y})
data.head()


,Previous Week Load,Present Week Load
0,5551.82208,5346.52812
1,4983.17184,5138.89596
2,4888.39680,4940.77248
3,5072.95872,4766.18688
4,5196.25980,5020.11540


In [83]:
maxX = np.max(data['Previous Week Load'])
minX = np.min(data['Previous Week Load'])
maxY = np.max(data['Present Week Load'])
minY = np.min(data['Present Week Load'])

data['Previous Week Load'] = (data['Previous Week Load'] - minX) / (maxX - minX)
data['Present Week Load'] = (data['Present Week Load'] - minY) / (maxY - minY)

In [84]:
x = data['Previous Week Load']
y = data['Present Week Load']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [85]:
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.5
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6
# Loop for epochs
for _ in range(epochs):  
    for i in range(trainX.shape[0]):
        gradM = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c) * trainX.iloc[i]
        gradC = -1 * (trainY.iloc[i] - m * trainX.iloc[i] - c)
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

# Print model parameters after training
print("value of m : ",m)
print("value of c : ",c)

value of m :  0.4857348341509555
value of c :  0.23135523213312134


In [86]:
train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]

# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

Training Error :
MAE : 737.027186961702
MSE : 891424.9426498839
RMSE : 944.1530292542009


In [87]:
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Testing Error :
MAE : 787.1173748633687
MSE : 968116.8376134088
RMSE : 983.9292848642167


In [88]:
x = float(input('give load of same hour in previous week: '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load of present hour :', prediction)

give load of same hour in previous week: 5000
Predicted load of present hour : 5428.769015117672
